<a href="https://colab.research.google.com/github/dcolinmorgan/grph/blob/main/colab_cucat_bench_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cu_cat demo-datasets benchmark

In [1]:
!pip install --extra-index-url=https://pypi.nvidia.com cuml-cu12==23.12.00 cudf-cu12==23.12.00 #cugraph-cu11 pylibraft_cu11 raft_dask_cu11 dask_cudf_cu11 pylibcugraph_cu11 pylibraft_cu11
!pip install git+https://github.com/graphistry/pygraphistry.git@feat/gpu-featurization
# !pip install git+https://github.com/graphistry/pygraphistry.git@dev/depman_gpufeat
# !pip install --upgrade git+https://github.com/graphistry/cu-cat.git@master
# !git clone https://github.com/graphistry/cu-cat.git


!pip3 install --upgrade cu_cat>=0.7.32 --verbose

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.2/955.2 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.4/582.4 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import cuml
cuml.__version__

'23.12.00'

In [3]:
!echo "0.08.02" > VERSION.txt
!cp VERSION.txt /usr/local/lib/python3.10/dist-packages/cu_cat/

In [4]:
import os
from collections import Counter
import cProfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pstats import Stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 200)
import cudf,cuml,cupy

In [5]:
# from importlib import reload
# reload(cu_cat)
# os.chdir('cu-cat')
import cu_cat
# os.chdir('..')
# print(cu_cat.__version__)

In [6]:
import graphistry
graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username='dcolinmorgan', password='fXjJnkE3Gik6BWy') ## key id, secret key
graphistry.__version__

'0.33.0+96.ga654f9f'

In [7]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
# !pip install dirty_cat
# import dirty_cat

In [8]:
import logging
logger = logging.getLogger()
logging.basicConfig(level=logging.DEBUG)
!pip install dirty_cat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 3.3 MB/s eta 0:00:00


In [9]:
def prepro(X):
  obj_col=X.select_dtypes(include=['object']).columns
  for i in obj_col:
      X[i]=X[i].replace('nan',np.nan).fillna('0o0o0')
      X[i]=X[i].str.rjust(4,'0')
      X[i]=X[i].str.replace('.', 'dot', regex=False) #for IP addresses
  num_col=X.select_dtypes(include=['int64','float64']).columns
  for i in num_col:
      X[i]=X[i].fillna(0)
  return X

In [10]:
!nvidia-smi

Thu Jan  4 08:57:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0              26W /  70W |    103MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## ask HN

In [11]:
# # get the data top 3000 posts on Hacker News
askHN = pd.read_csv('https://storage.googleapis.com/cohere-assets/blog/text-clustering/data/askhn3k_df.csv', index_col=0)

askHN = askHN.sample(100,replace=False) # set smaller if you want to test a minibatch
# df = df[['text','title']]
# df['title'] = df['title'].sample(frac=1).values

In [18]:
# for i in askHN.columns:
#     askHN[i]=askHN[i].str.rjust(4,'0')#apply(lambda x: str(x).zfill(4))
# # # bb=pd.DataFrame(bb)
# # # bb['len']=bb.by.str.len()
# # # bb.sort_values('len')
# # # # bb.iloc[497]
# # import cudf
# askHN#.by.str.len().sort_values()

something strange happening with degree category count for datasets with lots of text

In [26]:
g = graphistry.nodes(prepro(askHN[['text','title']])) # 'id','parent']]))#, \
        # 'id', 'parent']]))#, 'descendants', 'ranking', 'deleted']]))

pr = cProfile.Profile()
pr.enable()


# g1a = g.featurize(feature_engine='cu_cat',memoize=False)
g1aa = g.umap(engine='cuml',memoize=False,feature_engine='cu_cat')

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_cucat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         535372 function calls (529177 primitive calls) in 2.034 seconds

   Ordered by: internal time
   List reduced from 2321 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.785    0.262    0.785    0.262 {built-in method cupy_backends.cuda.libs.nvrtc.compileProgram}
        2    0.461    0.230    0.461    0.230 {built-in method cupy.cuda.jitify.jitify}
     1188    0.155    0.000    0.301    0.000 /usr/lib/python3.10/contextlib.py:76(inner)
        2    0.038    0.019    0.038    0.019 {method 'read' of '_io.BufferedReader' objects}
     31/6    0.030    0.001    1.617    0.270 /usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:138(wrapper)
99556/99388    0.023    0.000    0.038    0.000 {built-in method builtins.isinstance}
        9    0.019    0.002    0.127    0.014 /usr/local/lib/python3.10/dist-packages/cudf/core/frame.py:416(_to_array)
     4669    0.015    0.000    0.035    0.000 /us

In [28]:
g1aa.plot()

In [27]:
# !pip install dirty_cat
g = graphistry.nodes(prepro(askHN[['text','title']]))
pr = cProfile.Profile()
pr.enable()


# g1b = g.featurize(feature_engine='dirty_cat',memoize=False)
g1bb = g.umap(engine='cuml',memoize=False,feature_engine='dirty_cat') ## crashes colab

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('aHN_umap_dirtycat_3k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         4517897 function calls (4512835 primitive calls) in 11.920 seconds

   Ordered by: internal time
   List reduced from 2824 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       64    3.151    0.049    6.877    0.107 /usr/local/lib/python3.10/dist-packages/dirty_cat/_gap_encoder.py:1006(_multiplicative_update_h)
603064/602542    1.808    0.000    1.901    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        6    1.368    0.228    1.368    0.228 {built-in method cupy_backends.cuda.libs.nvrtc.compileProgram}
   567042    0.538    0.000    2.413    0.000 <__array_function__ internals>:177(dot)
        2    0.501    0.251    0.501    0.251 {built-in method cupy.cuda.jitify.jitify}
       16    0.432    0.027    1.080    0.067 /usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1258(_count_vocab)
    13268    0.318    0.000    0.426    0.000 /usr/local/lib/python3.10/dist-

In [29]:
g1bb.plot()

## 20newsgroups

In [20]:
from sklearn.datasets import fetch_20newsgroups
# newsgroups = fetch_20newsgroups()#categories=categories)
n_samples = 400
# n_features = 1000
# n_components = 10
# n_top_words = 20
# batch_size = 128
# init = "nndsvda"

news, _ = fetch_20newsgroups(
    shuffle=True,
    random_state=1,
    remove=("headers", "footers", "quotes"),
    return_X_y=True,
)

news = news[:n_samples]
news=pd.DataFrame(news)

In [ ]:
# !pip install dirty_cat
# from dirty_cat import TableVectorizer
# tv = TableVectorizer()
# tv.fit_transform(news)
# tv.get_feature_names()

In [ ]:
# from cu_cat import TableVectorizer, GapEncoder
# tv = TableVectorizer()
# tv.fit_transform(news)
# tv.get_feature_names()

In [30]:
g = graphistry.nodes(prepro(news))#.replace('nan',np.nan).fillna('0o0o0'))
pr = cProfile.Profile()
pr.enable()


# g2 = g.featurize(feature_engine='cu_cat',memoize=False)
g4a = g.umap(engine='cuml',feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         802602 function calls (797906 primitive calls) in 1.453 seconds

   Ordered by: internal time
   List reduced from 2235 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2057    0.402    0.000    0.642    0.000 /usr/lib/python3.10/contextlib.py:76(inner)
     3272    0.193    0.000    0.199    0.000 /usr/local/lib/python3.10/dist-packages/numba/misc/dummyarray.py:110(compute_index)
     29/6    0.042    0.001    0.739    0.123 /usr/local/lib/python3.10/dist-packages/cuml/internals/api_decorators.py:138(wrapper)
        2    0.037    0.019    0.037    0.019 {method 'read' of '_io.BufferedReader' objects}
149377/149253    0.036    0.000    0.059    0.000 {built-in method builtins.isinstance}
     8029    0.026    0.000    0.062    0.000 /usr/local/lib/python3.10/dist-packages/cudf/core/dtypes.py:969(is_categorical_dtype)
        7    0.025    0.004    0.394    0.056 /usr/local/lib/python3.10/dist-packages/cudf/core/frame.

In [31]:
g4a.plot()

In [54]:
g = graphistry.nodes(news)
pr = cProfile.Profile()
pr.enable()


# g2 = g.featurize(feature_engine='dirty_cat',memoize=False)
g4b = g.umap(engine='cuml',feature_engine='dirty_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('20news_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         16382190 function calls (16376295 primitive calls) in 53.312 seconds

   Ordered by: internal time
   List reduced from 2320 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      236   18.525    0.078   40.050    0.170 /usr/local/lib/python3.10/dist-packages/dirty_cat/_gap_encoder.py:1006(_multiplicative_update_h)
2683317/2683031   13.630    0.000   15.262    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
  2561136    2.594    0.000   16.537    0.000 <__array_function__ internals>:177(dot)
        8    2.139    0.267    5.112    0.639 /usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1258(_count_vocab)
       20    1.392    0.070    3.643    0.182 /usr/local/lib/python3.10/dist-packages/dirty_cat/_gap_encoder.py:974(_multiplicative_update_w)
    54888    1.351    0.000    1.832    0.000 /usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:275(_ch

In [55]:
g4b.plot()

## redteam

In [32]:
df = pd.read_csv('https://gist.githubusercontent.com/silkspace/c7b50d0c03dc59f63c48d68d696958ff/raw/31d918267f86f8252d42d2e9597ba6fc03fcdac2/redteam_50k.csv', index_col=0)
red_team = pd.read_csv('https://gist.githubusercontent.com/silkspace/5cf5a94b9ac4b4ffe38904f20d93edb1/raw/888dabd86f88ea747cf9ff5f6c44725e21536465/redteam_labels.csv', index_col=0)
df['feats'] = df.src_computer + ' ' + df.dst_computer + ' ' + df.auth_type + ' ' + df.logontype
df['feats2'] = df.src_computer + ' ' + df.dst_computer
ndf = df.drop_duplicates(subset=['feats'])
tdf = pd.concat([red_team.reset_index(), ndf.reset_index()])
tdf['node'] = range(len(tdf))

In [33]:
%%time
# process = True  # set to false after it's run for ease of speed
pr = cProfile.Profile()
pr.enable()
g = graphistry.nodes(tdf, 'node')
g2a = g.umap(X=['feats'],
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='cu_cat',
            memoize=False,
            engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         1225352 function calls (1204056 primitive calls) in 52.774 seconds

   Ordered by: internal time
   List reduced from 2680 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3   18.610    6.203   18.610    6.203 /usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:523(in1d)
        8   15.074    1.884   19.962    2.495 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:1242(_multiplicative_update_h_smallfast)
       45    9.832    0.218    9.832    0.218 {built-in method cupy_backends.cuda.libs.nvrtc.compileProgram}
        7    1.909    0.273    2.193    0.313 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:273(_get_H)
     3590    1.698    0.000    1.770    0.000 /usr/local/lib/python3.10/dist-packages/rmm/allocators/cupy.py:23(rmm_cupy_allocator)
        5    0.928    0.186    0.928    0.186 {built-in method gc.collect}
        5    0.510    0.102    2.532    0.506 /usr/local/

In [47]:
g2a.plot()

In [34]:
%%time
pr = cProfile.Profile()
pr.enable()
g = graphistry.nodes(tdf, 'node')
g2b = g.umap(X=['feats'],
            min_words=1000000, # force high so that we don't use Sentence Transformers
            cardinality_threshold=4, # set low so we force topic model
            n_topics=32, # number of topics
            use_scaler=None,
            feature_engine='dirty_cat',
            memoize=False,
            engine='cuml',
            use_scaler_target=None
           )

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('redteam_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         26809875 function calls (26613122 primitive calls) in 98.518 seconds

   Ordered by: internal time
   List reduced from 2240 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1054   20.047    0.019   59.218    0.056 /usr/local/lib/python3.10/dist-packages/dirty_cat/_gap_encoder.py:1006(_multiplicative_update_h)
        3   17.386    5.795   17.386    5.795 /usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:523(in1d)
3337142/3336917   14.679    0.000   32.405    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
      775    4.901    0.006   14.259    0.018 /usr/local/lib/python3.10/dist-packages/dirty_cat/_gap_encoder.py:974(_multiplicative_update_w)
   272948    4.805    0.000   10.109    0.000 /usr/local/lib/python3.10/dist-packages/scipy/sparse/_sputils.py:147(get_index_dtype)
   134890    3.919    0.000   12.833    0.000 /usr/local/lib/python3.10/dist-packages/scipy/sparse/_c

In [ ]:
g2b.plot()

## CTU-13 malware dataset

In [35]:
edf = pd.read_csv('https://gist.githubusercontent.com/silkspace/33bde3e69ae24fee1298a66d1e00b467/raw/dc66bd6f1687270be7098f94b3929d6a055b4438/malware_bots.csv', index_col=0)
T = edf.Label.apply(lambda x: True if 'Botnet' in x else False)
bot = edf[T]
nbot = edf[~T]
print(f'Botnet abundance: {100*len(bot)/len(edf):0.2f}%')# so botnet traffic makes up a tiny fraction of total

# let's balance the dataset in a 10-1 ratio, for speed and demonstrative purposes
negs = nbot.sample(10*len(bot))
edf = pd.concat([bot, negs])  # top part of arrays are bot traffic, then all non-bot traffic
edf = edf.drop_duplicates()

# some useful indicators for later that predict Botnet as Bool and Int
Y = edf.Label.apply(lambda x: 1 if 'Botnet' in x else 0)  # np.array(T)

# Later we will use and exploit any meaning shared between the labels in a latent distribution

# add it to the dataframe
edf['bot'] = Y

# name some columns for edges and features
src = 'SrcAddr'
dst = 'DstAddr'
good_cols_with_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes', src, dst]

good_cols_without_edges = ['Dur', 'Proto', 'Sport',
       'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes']

## some encoding parameters
n_topics = 20
n_topics_target = 7



Botnet abundance: 0.69%


In [36]:
# # edf.DstAddr.str.len().sort_values()#.dropna()

# edf.DstAddr=edf.DstAddr.str.replace('.', 'dot', regex=False)
# edf.SrcAddr=edf.SrcAddr.str.replace('.', 'dot', regex=False)

In [37]:
# edf.DstAddr

In [38]:
# lets umap the data
g = graphistry.nodes(edf[['SrcAddr','DstAddr']])#.apply(lambda x: str(x).zfill(4)))#, src, dst)
pr = cProfile.Profile()
pr.enable()

g3a = g.umap(engine='cuml',memoize=False,feature_engine='cu_cat')

# g3a = g.umap(kind='edges',
#             # X=good_cols_with_edges,
#             # y = ['bot'],
#             use_scaler='quantile',
#             use_scaler_target=None,
#             cardinality_threshold=20,
#             cardinality_threshold_target=2,
#             n_topics=n_topics,
#             feature_engine='cu_cat',
#             engine='cuml',
#             memoize=False,
#             n_topics_target=n_topics_target,
#             n_bins=n_topics_target,
#             metric='euclidean',
#             n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_cucat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


         1465360 function calls (1428219 primitive calls) in 10.429 seconds

   Ordered by: internal time
   List reduced from 2608 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    1.871    0.187    1.871    0.187 {built-in method gc.collect}
        4    1.410    0.353    1.410    0.353 {built-in method cupy_backends.cuda.libs.nvrtc.compileProgram}
       16    1.085    0.068    2.122    0.133 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:1242(_multiplicative_update_h_smallfast)
       14    0.893    0.064    1.037    0.074 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:273(_get_H)
        6    0.893    0.149    0.893    0.149 /usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:523(in1d)
        2    0.729    0.364    0.729    0.364 {built-in method cupy.cuda.jitify.jitify}
     7141    0.471    0.000    0.551    0.000 /usr/local/lib/python3.10/dist-packages/rmm/allocators/

In [39]:
g3a.plot()

In [40]:
# lets umap the data
g = graphistry.nodes(edf[['SrcAddr','DstAddr']])#.apply(lambda x: str(x).zfill(4)))#, src, dst)
pr = cProfile.Profile()
pr.enable()

# !pip install dirty_cat
g3b = g.umap(engine='cuml',memoize=False,feature_engine='dirty_cat')

# g3b = g.umap(kind='edges',
#             X=good_cols_with_edges,
#             y = ['bot'],
#             use_scaler='quantile',
#             use_scaler_target=None,
#             cardinality_threshold=20,
#             cardinality_threshold_target=2,
#             n_topics=n_topics,
#             feature_engine='dirty_cat',
#             engine='cuml',
#             memoize=False,
#             n_topics_target=n_topics_target,
#             n_bins=n_topics_target,
#             metric='euclidean',
#             n_neighbors=12)
pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('ctu13_umap_dirtycat.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)


         9593286 function calls (9485452 primitive calls) in 38.653 seconds

   Ordered by: internal time
   List reduced from 2277 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      823    9.704    0.012   27.353    0.033 /usr/local/lib/python3.10/dist-packages/dirty_cat/_gap_encoder.py:1006(_multiplicative_update_h)
1345059/1344539    7.436    0.000    8.261    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
    97066    2.082    0.000    4.498    0.000 /usr/local/lib/python3.10/dist-packages/scipy/sparse/_sputils.py:147(get_index_dtype)
  1140512    1.748    0.000    9.026    0.000 <__array_function__ internals>:177(dot)
    46888    1.747    0.000    5.449    0.000 /usr/local/lib/python3.10/dist-packages/scipy/sparse/_compressed.py:135(check_format)
      780    1.539    0.002    6.661    0.009 /usr/local/lib/python3.10/dist-packages/dirty_cat/_gap_encoder.py:974(_multiplicative_update_w)
   194

In [41]:
g3b.plot()

## winlogs

In [42]:
!wget -nc https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet
winlogs=pd.read_parquet('part.88.parquet')
winlogsA=winlogs.sample(10000,replace=False)#.iloc[:,:10]
# winlogsA=winlogsA#[['LogonID','UserName','LogHost']]#,'DomainName','LogonType','SubjectLogonID','Status','Destination','ServiceName']]#,'EventID']]#.convert_dtypes() #.replace('nan',np.nan).fillna('0o0o0')


--2024-01-04 09:11:21--  https://www.dropbox.com/s/31dx1g6g59exoc3/part.88.parquet
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/31dx1g6g59exoc3/part.88.parquet [following]
--2024-01-04 09:11:21--  https://www.dropbox.com/s/raw/31dx1g6g59exoc3/part.88.parquet
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc78e5a2f6077658556ac136b6e9.dl.dropboxusercontent.com/cd/0/inline/CKsNFkozHodCAdfomOBAIFwaheTTnACRb78btmUVMM9NLRi434-7BYzkIu7M0Tuwrt8VcK296_MKgg2GIjJtCaHX2h0_b1EufsEns5QNgJ-0SEjZQH0LMBhuwolCTW3ToeY/file# [following]
--2024-01-04 09:11:22--  https://uc78e5a2f6077658556ac136b6e9.dl.dropboxusercontent.com/cd/0/inline/CKsNFkozHodCAdfomOBAIFwaheTTnACRb78btmUVMM9NLRi434-7BYzkIu7M0Tuwrt8VcK296_MKgg2GIjJtCaHX2h0_b1EufsEns5QNgJ-0SEjZ

In [43]:
# import cupy as cp
# del g,g2,g3
# cp._default_memory_pool.free_all_blocks()

In [44]:
def prepro(X):
  obj_col=X.select_dtypes(include=['object']).columns
  for i in obj_col:
      X[i]=X[i].replace('nan',np.nan).fillna('0o0o0')
      X[i]=X[i].str.rjust(4,'0')
      X[i]=X[i].str.replace('.', 'dot', regex=False) #for IP addresses
  num_col=X.select_dtypes(include=['int64','float64']).columns
  for i in num_col:
      X[i]=X[i].fillna(0)
  return X

In [45]:
g = graphistry.nodes(prepro(winlogsA))#.replace('nan',np.nan).fillna('0o0o0'))
pr = cProfile.Profile()
pr.enable()

# g5a = g.featurize(feature_engine='cu_cat',memoize=False)
g5aa = g.umap(engine='cuml',feature_engine='cu_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize_cucat_20k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         6680449 function calls (6521880 primitive calls) in 73.071 seconds

   Ordered by: internal time
   List reduced from 2733 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       63   26.853    0.426   32.724    0.519 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:1242(_multiplicative_update_h_smallfast)
       28   12.253    0.438   12.254    0.438 /usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:523(in1d)
       35    7.296    0.208    7.296    0.208 {built-in method gc.collect}
       63    4.480    0.071    5.204    0.083 /usr/local/lib/python3.10/dist-packages/cu_cat/_gap_encoder.py:273(_get_H)
    24227    3.899    0.000    4.246    0.000 /usr/local/lib/python3.10/dist-packages/rmm/allocators/cupy.py:23(rmm_cupy_allocator)
11031/10914    2.351    0.000    4.139    0.000 /usr/lib/python3.10/contextlib.py:76(inner)
        6    1.839    0.307    1.839    0.307 {built-in method cupy_backen

In [49]:
g5aa.plot()

In [46]:
g = graphistry.nodes(prepro(winlogsA))
pr = cProfile.Profile()
pr.enable()

# g5b = g.featurize(feature_engine='dirty_cat',memoize=False)
g5bb = g.umap(engine='cuml',feature_engine='dirty_cat',memoize=False)

pr.disable()
stats = Stats(pr)
stats.sort_stats('tottime').print_stats(20)

with open('winlogs_featurize_dirtycat_20k.txt', 'w') as stream:
    stats = Stats(pr,stream=stream)
    stats.sort_stats('tottime').print_stats(20)

         38531295 function calls (37713709 primitive calls) in 141.697 seconds

   Ordered by: internal time
   List reduced from 2331 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
4487201/4485112   28.655    0.000   35.024    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
     3918   26.651    0.007   75.753    0.019 /usr/local/lib/python3.10/dist-packages/dirty_cat/_gap_encoder.py:1006(_multiplicative_update_h)
     3555   17.429    0.005   45.847    0.013 /usr/local/lib/python3.10/dist-packages/dirty_cat/_gap_encoder.py:974(_multiplicative_update_w)
   384302    6.259    0.000   13.481    0.000 /usr/local/lib/python3.10/dist-packages/scipy/sparse/_sputils.py:147(get_index_dtype)
       27    5.238    0.194    5.238    0.194 /usr/local/lib/python3.10/dist-packages/numpy/lib/arraysetops.py:523(in1d)
   184669    5.099    0.000   16.633    0.000 /usr/local/lib/python3.10/dist-packages/scipy/sparse/_

In [50]:
g5bb.plot()